In command line:

#launch sqlite command line tool & create database called infoDB
sqlite3 infoDB   

# tab-delimted file
.separator "\t"

# some times mode need to be updated
.mode csv

# create new table called "info" w/ the content from our tsv
.import info.csv info

In [24]:
import sqlite3
import random

In [53]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def select_by_slot(conn, slot_name, slot_value):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM info
                WHERE {slot_name}='{slot_value}'""")

    rows = cur.fetchall()

    if len(list(rows)) < 1:
        print("There is no info matching your query.")
    else:
        for row in random.sample(rows, 1):
            print(f"You can go through a {row[4].lower()} about '{row[0]}' by {row[1]}. You can find it at {row[2]}.")

In [68]:
select_by_slot(create_connection("infoDB"), slot_name="Type", slot_value="Work")

You can go through a work about 'Django/Plotly dashsensors data' by Anonymous Budhathoki. You can find it at https://www.budhathokisagar.com.np/works/1/.


In [ ]:
# This files contains your custom actions which can be used to run
# custom Python code.
#
# See this guide on how to implement these action:
# https://rasa.com/docs/rasa/custom-actions


# This is a simple example for a custom action which utters "Hello World!"

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher

import sqlite3
import random


class QueryInfoType(Action):

    def name(self) -> Text:
        return "query_info_type"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        conn = create_connection(self, db_file="../info_db/info")

        slot_value = tracker.get_slot("info_type")
        slot_name = "Type"

        get_query_results = select_by_slot(self, conn=conn,
            slot_name=slot_name,slot_value=slot_value)
        dispatcher.utter_message(text=get_query_results)

        return []

    def create_connection(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
        except Error as e:
            print(e)

        return conn

    def select_by_slot(conn, slot_name, slot_value):
        """
        Query all rows in the tasks table
        :param conn: the Connection object
        :return:
        """
        cur = conn.cursor()
        cur.execute(f"""SELECT * FROM info
                    WHERE {slot_name}='{slot_value}'""")

        rows = cur.fetchall()

        if len(list(rows)) < 1:
            return[("There are no info matching your query.")]
        else:
            for row in random.sample(rows, 1):
                return[print(f"Try the {(row[4]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")]

In [ ]:
# from: https://techoverflow.net/2019/10/14/how-to-list-tables-in-sqlite3-database-in-python/
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

In [ ]:

conn = QueryResourceType.create_connection("../infoDB")
print(tables_in_sqlite_db(conn))

some testing for rasa project:
   rasa data validate
   rasa test nlu
   rasa data split nlu
   rasa test nlu -u train_test_split/test_data.yml
   rasa train nlu
   rasa test nlu -m models/nlu-20210731-022226.tar.gz -u train_test_split/test_data.yml 